In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paperlist/Authorship-Data-v1.csv


In [2]:
df = pd.read_csv('/kaggle/input/paperlist/Authorship-Data-v1.csv')
df.head()

,Authors,Title,Year,Source title,Cited by,DOI,Abstract,Document Type,Open Access
0,Chandrika C.P.; Kallimani J.S.,Instance Based Authorship Attribution for Kann...,2022,Lecture Notes in Electrical Engineering,2,10.1007/978-981-19-2281-7_51,Authorship Attribution is the task of identify...,Conference paper,NaN
1,NaN,Erratum: Brian Vickers (ed.). The Collected Wo...,2021,Review of English Studies,0,10.1093/res/hgy064,"The following changes have been made: Page 1, ...",Erratum,All Open Access; Bronze Open Access
2,NaN,9th Brazilian Conference on Intelligent System...,2020,Lecture Notes in Computer Science (including s...,0,NaN,The proceedings contain 91 papers. The special...,Conference review,NaN
3,Plecháč P.; Birnbaum D.J.,Rhythm-based authorship recognition in syllabi...,2023,"Literatura: Teoria, Historia, Critica",0,10.15446/lthc.v25n2.108502,This contribution explores the extent to which...,Article,NaN
4,Khomytska I.; Teslyuk V.; Bazylevych I.; Kordi...,Machine Learning and Classical Methods Combine...,2022,CEUR Workshop Proceedings,3,NaN,The novelty of the research is an offered comb...,Conference paper,NaN


## FILTER DATA FRAME

In [3]:
df = df[['Abstract']]
df.head(2)

,Abstract
0,Authorship Attribution is the task of identify...
1,"The following changes have been made: Page 1, ..."


In [4]:
rand_index = np.random.randint(0, len(df))
df['Abstract'][rand_index]

'The classification of textual data often yields important information. Most classifiers work in a closed world setting where the classifier is trained on a known corpus, and then it is tested on unseen examples that belong to one of the classes seen during training. Despite the usefulness of this design, often there is a need to classify unseen examples that do not belong to any of the classes on which the classifier was trained. This paper describes the open set scenario where unseen examples from previously unseen classes are handled while testing. This further examines a process of enhanced open set classification with a deep neural network that discovers new classes by clustering the examples identified as belonging to unknown classes, followed by a process of retraining the classifier with newly recognized classes. Through this process the model moves to an incremental learning model where it continuously finds and learns from novel classes of data that have been identified autom

In [5]:
def clean_text(text):
    """
    Lowercase the input text and remove extra spaces.
    
    Args:
        text (str): Input text.
        
    Returns:
        str: Cleaned text with lowercase and no extra spaces.
    """
    # Lowercase the text and split it into words
    words = text.lower().split()
    
    # Join the words back to form a string without extra spaces
    cleaned_text = ' '.join(words)
    
    return cleaned_text

In [6]:
from tqdm import tqdm
tqdm.pandas()
df.Abstract = df.Abstract.progress_apply(clean_text)

100%|██████████| 299/299 [00:00<00:00, 24487.86it/s]


In [7]:
df['Abstract'][0]

'authorship attribution is the task of identifying a true author of a given text from a set of suspected authors stylometry features play a vital role in recognizing the right author, it includes lexical and syntactic features. n-gram is one of the popular techniques used to extract syntactic features from the text. the main objective of this work is to use both lexical and syntactic features on a kannada text and compare the performance of both approaches using different machine learning algorithms. the kannada language is spoken by the indian southern state karnataka. even though we can see major works in text processing, authorship attribution is in a tender state. researches have been carried out on handwritten kannada documents but not on digital text. char n-gram, word n-gram and the combination of these two known as amalgamation technique are used as syntactic features to extract the writing style of an author. the results show that support vector machine algorithm outperform wi

In [21]:
sort_dict = {
    3 : ['literature', 'authorship', 'attribution', 'stylometric', 'style', 'pattern'],
    2 : ['novel', 'poem', 'poet', 'novelist', 'nonfiction'],
    1 : ['fiction', 'poetry', 'drama', 'play', 'dramatist', 'playwright', 'feel', 'movie', 'newspaper']
}


In [22]:
sort_dict[3]

['literature', 'authorship', 'attribution', 'stylometric', 'style', 'pattern']

In [25]:
rand_index = np.random.randint(0, len(df))
test = df['Abstract'][rand_index]
w = 0
for word in tqdm(test.split()):
    if word in sort_dict[3]:
        w += 3
    elif word in sort_dict[2]:
        w += 2
    elif word in sort_dict[1]:
        w += 1
print(f'Weight of Index {rand_index} = {w}')

100%|██████████| 329/329 [00:00<00:00, 662470.48it/s]

Weight of Index 34 = 18


### Implement a function

In [31]:
def generate_weight(text):
    '''
    Produce a weight according to some predefined factors.
    
    Args:
        text (str): Input text.
        
    Returns:
        weight (int): Weight of the text.
    '''
    w = 0
    for word in text.split():
        if word in sort_dict[3]:
            w += 3
        elif word in sort_dict[2]:
            w += 2
        elif word in sort_dict[1]:
            w += 1
    return w

In [32]:
generate_weight(df.Abstract[0])

16

### Apply it in the whole dataframe

In [33]:
tqdm.pandas()
df['weight'] = df['Abstract'].progress_apply(generate_weight)

100%|██████████| 299/299 [00:00<00:00, 9578.45it/s]


In [34]:
df.head()

,Abstract,weight
0,authorship attribution is the task of identify...,16
1,"the following changes have been made: page 1, ...",12
2,the proceedings contain 91 papers. the special...,6
3,this contribution explores the extent to which...,15
4,the novelty of the research is an offered comb...,9


In [37]:
sorted_df = df.sort_values(by = 'weight', ascending = False)
sorted_df.head(2)

,Abstract,weight
257,problem statement: stylometric authorship attr...,44
138,authorship attribution aims at identifying the...,38


In [38]:
sorted_df.shape

(299, 2)

In [47]:
fdf = sorted_df[:100]
fdf.head()

,Abstract,weight
257,problem statement: stylometric authorship attr...,44
138,authorship attribution aims at identifying the...,38
194,stylometric authorship attribution is one of t...,36
39,open-source software is open to anyone by desi...,35
152,the style followed by authors can be thought o...,34


In [48]:
data = pd.read_csv('/kaggle/input/paperlist/Authorship-Data-v1.csv')
data.head(2)

,Authors,Title,Year,Source title,Cited by,DOI,Abstract,Document Type,Open Access
0,Chandrika C.P.; Kallimani J.S.,Instance Based Authorship Attribution for Kann...,2022,Lecture Notes in Electrical Engineering,2,10.1007/978-981-19-2281-7_51,Authorship Attribution is the task of identify...,Conference paper,NaN
1,NaN,Erratum: Brian Vickers (ed.). The Collected Wo...,2021,Review of English Studies,0,10.1093/res/hgy064,"The following changes have been made: Page 1, ...",Erratum,All Open Access; Bronze Open Access


In [49]:
fndf = data.loc[fdf.index]
fndf['Weight'] = fdf['weight']
fndf.head()

,Authors,Title,Year,Source title,Cited by,DOI,Abstract,Document Type,Open Access,Weight
257,Mustafa T.K.; Mustapha N.; Azmi M.A.; Sulaiman...,Dropping down the maximum item set: Improving ...,2010,Journal of Computer Science,4,10.3844/jcssp.2010.235.243,Problem statement: Stylometric authorship attr...,Article,All Open Access; Green Open Access; Hybrid Gol...,44
138,Sarwar R.; Yu C.; Tungare N.; Chitavisutthivon...,An effective and scalable framework for author...,2018,IEEE Access,12,10.1109/ACCESS.2018.2869198,Authorship attribution aims at identifying the...,Article,All Open Access; Gold Open Access; Green Open ...,38
194,Jamil M.T.; Mustafa T.K.,Ranking attribution: A novel method for stylom...,2018,International Journal of Advanced Computer Sci...,1,10.14569/IJACSA.2018.090709,Stylometric Authorship attribution is one of t...,Article,All Open Access; Gold Open Access; Green Open ...,36
39,Matyukhina A.; Stakhanova N.; Preda M.D.; Perl...,Adversarial authorship attribution in open-sou...,2019,CODASPY 2019 - Proceedings of the 9th ACM Conf...,2,10.1145/3292006.3300032,Open-source software is open to anyone by desi...,Conference paper,NaN,35
152,Neme A.; Pulido J.R.G.; Muñoz A.; Hernández S....,Stylistics analysis and authorship attribution...,2015,Neurocomputing,17,10.1016/j.neucom.2014.03.064,The style followed by authors can be thought o...,Article,NaN,34


In [50]:
fndf.to_csv('/kaggle/working/reduced.csv', index = True)

In [51]:
fndf.shape

(100, 10)

In [52]:
fndf['Abstract'][0]

'Authorship Attribution is the task of identifying a true author of a given text from a set of suspected authors stylometry features play a vital role in recognizing the right author, it includes lexical and syntactic features. N-gram is one of the popular techniques used to extract syntactic features from the text. The main objective of this work is to use both lexical and syntactic features on\xa0a Kannada text and compare the performance of both approaches using different machine learning algorithms. The Kannada language is spoken by the Indian southern state Karnataka. Even though we can see major works in text processing, Authorship Attribution is in a tender state. Researches have been carried out on handwritten Kannada documents but not on digital text. Char n-gram, word n-gram and the combination of these two known as Amalgamation technique are used as syntactic features to extract the writing style of an author. The results show that Support Vector Machine algorithm outperform